# Sentiment Analysis

In [1]:
!pip install -U nltk scikit-learn pandas matplotlib numpy wordcloud

Requirement already up-to-date: scikit-learn in c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages (0.21.3)
Requirement already up-to-date: matplotlib in c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages (3.1.1)
Requirement already up-to-date: numpy in c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages (1.17.0)
Requirement already up-to-date: wordcloud in c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages (1.5.0)
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449382 sha256=6ac15f1e836a4f87d44c7a719079e4fd928d324f481f6e3cbaa99aa2fac9bd1f
  Stored in directory: C:\Users\a.lima.laurentino\AppData\Local\pip\Cache\wheels\96\86\f6\68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.4.3
    Uninstalling nltk-3.4.3:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\program files (x86)\\microsoft visual studio\\shared\\anaconda3_64\\lib\\site-packages\\nltk-3.4.3.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Read the csv

In [2]:
import pandas as pd

reviews = pd.concat([pd.read_csv('./files/reviews_en-pt_1.csv'), pd.read_csv('./files/reviews_en-pt_2.csv')], join='outer')
reviews['sentiment'] = reviews['sentiment'].map({'neg': 0, 'pos': 1})
reviews.drop(columns=["text_pt", "id"], inplace=True)
reviews.head(10)

,text_en,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures movies is not a fitting word fo...,0
5,"A funny thing happened to me while watching ""M...",0
6,This German horror film has to be one of the w...,0
7,"Being a long-time fan of Japanese film, I expe...",0
8,"""Tokyo Eyes"" tells of a 17 year old Japanese g...",0
9,Wealthy horse ranchers in Buenos Aires have a ...,0


# Preprocessing

In [3]:
import re
from nltk.tokenize import WhitespaceTokenizer

### 1. Cleaning:
    Clear and get only the main part from the dataset
    Ex: remove the tags of the html.
    Ex: filter the texts in PDF and etc.

In [4]:
def clean(text):
    # Remove the HTML tags
    text = re.sub("<!--?.*?-->","",text)
    text = re.sub("<.*?>","",text)
    
    return text

### 2. Normalization:
    Remove the pontuation, tags, put everything in same case and etc.

In [5]:
def normalize(text):
    # Convert to lower case
    text = text.lower()
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ",text)
    
    text = text.replace('  ', ' ')
    
    return text

### 3. Tokenization:
    Split the text in words spliting by the whitespaces.

In [6]:
def tokenizer(text):    
    tokenizer = WhitespaceTokenizer()
    tokens = tokenizer.tokenize(text)
    
    return tokens

### 4. Stop Words:
    They are words witch don't get no one meaning, they are just used to complement the context,
    and to connect the terms.
    Ex: 'i', 'you', 'in', 'out', 'are', 'the'

In [7]:
from nltk.corpus import stopwords

# Remove the stop words, they are words witch don't give no one especific meaning
def remove_stopwords(tokens):
    return [w for w in tokens if w not in stopwords.words("english")]

### 5. Stemming:
    Takes of the variation of the words and remove the finally to combine than.
    Ex: 'change', 'changing', 'changes' => 'chang'


### 6. Lemmatization:
    Takes the variation of the same word and convert to the same one (Noun).
    Ex: 'is', 'were', 'was' => 'be'
    Ex: 'ones' => 'one'

    Part of Speech(PoS) (Verb):
    Ex: 'bored' => 'bore'
    Ex: 'stating' => 'start'

In [8]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# Remove the pural
# Remove the verb conjugation
def stem(words):
    return [PorterStemmer().stem(w) for w in words]

# Remove the personality
def lem(words):
    return [WordNetLemmatizer().lemmantize(w, ) for w in words]

### 7. Tag filtering
    Filter the words according with the sintaxe definition like a noun, verbs, adverbs e etc.

In [9]:
import nltk

# RB | RB | JJ | NN | NNP | JJ | JJS | IN | VB | VBZ | VBD | VBG

# IN = preposition/subordinating conjunction

# RB = adverb very, silently
# RBR = adverb, comparative better
# RBS = adverb, superlative best
# RP = particle give up

# IN = preposition/subordinating conjunction
# JJ = adjective ‘big’
# JJR = adjective, comparative ‘bigger’
# JJS = adjective, superlative ‘biggest’

# VB verb, base form take
# VBD verb, past tense took
# VBG verb, gerund/present participle taking
# VBN verb, past participle taken
# VBP verb, sing. present, non-3d take
# VBZ verb, 3rd person sing. present takes

# Filter using regular array
def filter_tokens(tokens):
    tags = [x[1] for x in nltk.pos_tag(list(tokens))]
    filters = ("RB", "RBR", "RBS", "RP", "JJ", "JJR", "JJS", "JJ", "VB")
    
    return [tokens[i] for i in range(len(tokens)) if tags[i] in filters]

# Bag of Words

In [10]:
all_words = " ".join(list(reviews.text_en[:1000]))

tokens = tokenizer(normalize(clean(all_words)))
print("> Tokenized!")

tokens = remove_stopwords(tokens)
print("> Removed the stop words!")

tokens = stem(tokens)
# tokens = lem(tokens)
print("> Merged the term by stem or lem!")

tokens = filter_tokens(tokens)
print("> Filtred by tags witch get more meaning!")

print("\nColection [:100]:\n")
print(tokens[:100])

> Tokenized!
> Removed the stop words!
> Merged the term by stem or lem!
> Filtred by tags witch get more meaning!

Colection [:100]:

['far', 'longer', 'necessari', 'charact', 'forgotten', 'much', 'later', 'better', 'sign', 'win', 'final', 'half', 'kutcher', 'best', 'prior', 'keep', 'turn', 'major', 'realli', 'worth', 'tap', 'proven', 'mani', 'well', 'dont', 'bother', 'go', 'see', 'new', 'new', 'undercov', 'n', 'higher', 'friday', 'real', 'clich', 'still', 'wonder', 'alway', 'play', 'exact', 'charact', 'alien', 'ive', 'exact', 'irrit', 'least', 'alien', 'somewhat', 'gratifi', 'overal', 'second', 'better', 'see', 'practic', 'better', 'better', 'script', 'worth', 'decent', 'almost', 'refresh', 'close', 'first', 'hate', 'gun', 'go', 'alreadi', 'warehous', 'also', 'sadler', 'much', 'right', 'peopl', 'everywher', 'pretti', 'much', 'big', 'get', 'deserv', 'black', 'ugli', 'dead', 'stay', 'away', 'crap', 'instead', 'lest', 'real', 'even', 'write', 'song', 'mop', 'top', 'provok', 'social', 'm

### Get the frequency of the words

In [11]:
import nltk

frequency = nltk.FreqDist(tokens)

# Create the bag of words dataframe
bag_of_words = pd.DataFrame({"words": list(frequency.keys()), "frequency": list(frequency.values())})

# Order by the Frequency
bag_of_words.sort_values(by="frequency", ascending=False, inplace=True)
bag_of_words.reset_index(drop=True, inplace=True)

# Save the bag of words
bag_of_words.to_csv('./files/bag-of-words.csv', index=True)

print(f"Back of words size: {bag_of_words.shape[0]}")

print(bag_of_words.shape[0])
bag_of_words.head(5)

Back of words size: 5190
5190


,words,frequency
0,bad,648
1,even,633
2,good,532
3,movi,430
4,much,387


### Plot the frequency in Word Cloud

In [12]:
def to_single_str(words, frequency):
    words = list(words)
    frequency = list(frequency)
    
    return " ".join([(words[i] + " ") * frequency[i] for i in range(len(frequency))])

In [13]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

word_cloud = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(to_single_str(bag_of_words["words"], bag_of_words["frequency"]))

plt.figure(figsize=(13, 13))
plt.imshow(word_cloud)
plt.show()

<Figure size 1300x1300 with 1 Axes>

# Feature Selection

## TF-IDF

### Configs

In [14]:
# Load the dictinary
bag_of_words = pd.read_csv('./files/bag-of-words.csv')
bag_of_words_array = bag_of_words.words.values

# Get the inputs
reviews = pd.concat([pd.read_csv('./files/reviews_en-pt_1.csv'), pd.read_csv('./files/reviews_en-pt_2.csv')], join='outer')
reviews['sentiment'] = reviews['sentiment'].map({'neg': 0, 'pos': 1})
reviews.drop(columns=["text_pt", "id"], inplace=True)

inputs = reviews.text_en.values

### Implementation

In [15]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer


def tf_idf(txt, vocabulary=None):
    txt = list(txt)

    tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word', vocabulary=vocabulary)
    txt_transformed = tf.fit(txt).transform(txt)

    return pd.DataFrame(txt_transformed.toarray(), columns=tf.get_feature_names())

In [16]:
tfidf = tf_idf(inputs, bag_of_words_array)
tfidf.head(10)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\feature_extraction\text.py:1278: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


,bad,even,good,movi,much,get,well,first,better,ever,...,flail,copyright,kenni,yakkel,profan,loney,quench,outlin,choic,reliev
0,0.0,0.000000,0.00000,0.0,2.286652,0.000000,2.212528,0.000000,2.701125,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.00000,0.0,0.000000,0.000000,2.212528,0.000000,8.103376,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.00000,0.0,4.573304,2.337175,0.000000,2.398623,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.112969,0.00000,0.0,0.000000,0.000000,2.212528,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,1.97721,0.0,2.286652,0.000000,2.212528,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,1.97721,0.0,2.286652,0.000000,0.000000,0.000000,0.000000,2.657334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.00000,0.0,0.000000,4.674350,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,2.112969,1.97721,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modeling

In [17]:
from sklearn.model_selection import train_test_split

x = tfidf.values
y = [[x] for x in reviews.sentiment.values]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0, stratify = y)

### Dummy Classifier

In [18]:
from sklearn.dummy import DummyClassifier

model = DummyClassifier()
model.fit(x_train, y_train)

accuracy = model.score(x_test, y_test) * 100
print("Taxa de acerto do algoritimo de Base line: %.2f%%" % accuracy)

Taxa de acerto do algoritimo de Base line: 49.89%


### Linear SVC

In [19]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import numpy as np

np.random.seed(5)

# Test a linear model
model = LinearSVC()
model.fit(x_train, y_train)

accuracy = model.score(x_test, y_test) * 100
print("Linear SVC accuracy: %.2f%%" % accuracy)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Linear SVC accuracy: 79.22%
